In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


D:\PYFK\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
df = pd.read_csv('HousingData.csv')

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
columns = ['CRIM','ZN','INDUS','AGE','LSTAT']
for column in columns:
    df.fillna(df[column].mean(),axis = 0,inplace = True)
    
df['CHAS'].fillna(0,inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
fig, ax = plt.subplots(2, 7, figsize=(20, 10))
idx = 0
ax = ax.flatten()

for col, value in df.items():
    sns.boxplot(y=col, data=df, ax=ax[idx])
    idx += 1
plt.tight_layout(pad=0.5, w_pad=0.7, h_pad=5.0)

In [ ]:
for column, val in df.items():
        q1 = val.quantile(0.25)
        q3 = val.quantile(0.75)
        iqr = q3 - q1
        v_col = val[(val <= q1 - 1.5 * iqr) | (val >= q3 + 1.5 * iqr)]
        perc = np.shape(v_col)[0] * 100.0 / np.shape(df)[0]
        print("Column %s outliers = %.2f%%" % (column, perc))

In [ ]:
fig, ax = plt.subplots(2, 7, figsize=(20, 10))
idx = 0
ax = ax.flatten()

for col, value in df.items():
    sns.distplot(value, ax=ax[idx])
    idx += 1
plt.tight_layout(pad=0.5, w_pad=0.7, h_pad=5.0)

In [ ]:
cols = ['CRIM','ZN','TAX','B']
for col in cols:
    minimum = min(df[col])
    maximum = max(df[col])
    df[col] = (df[col] - minimum) / (maximum-minimum)

In [ ]:
fig, ax = plt.subplots(2, 7, figsize=(20, 10))
idx = 0
ax = ax.flatten()

for col, value in df.items():
    sns.distplot(value, ax=ax[idx])
    idx += 1
plt.tight_layout(pad=0.5, w_pad=0.7, h_pad=5.0)

In [ ]:
from sklearn import preprocessing
scalar = preprocessing.StandardScaler()

scaled_cols = scalar.fit_transform(df[cols])
scaled_cols = pd.DataFrame(scaled_cols,columns = cols)
scaled_cols.head()

In [ ]:
for col in cols:
    df[col] = scaled_cols[col]

In [ ]:
fig, ax = plt.subplots(2, 7, figsize=(20, 10))
idx = 0
ax = ax.flatten()

for col, value in df.items():
    sns.distplot(value, ax=ax[idx])
    idx += 1
plt.tight_layout(pad=0.5, w_pad=0.7, h_pad=5.0)

In [ ]:
plt.figure(figsize=(20,10))

corr = df.corr()
sns.heatmap(corr,annot = True,cmap = 'inferno')

In [ ]:
plt.figure(figsize=(20,10))
sns.regplot(y = df['MEDV'],x = df['LSTAT'])

In [ ]:
plt.figure(figsize=(20,10))
sns.regplot(y = df['MEDV'],x = df['RM'])

In [ ]:
X = df.drop(columns = ['MEDV','RAD'],axis = 1)
y = df['MEDV']

In [ ]:
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error

def train(model,X,y):
    x_train,x_test,y_train,y_test = train_test_split(X,y,random_state = 42)
    model.fit(x_train,y_train)
    
    pred = model.predict(x_test)
    
    cv_score = cross_val_score(model,X,y,scoring = 'neg_mean_squared_error',cv = 5)
    cv_score = np.abs(np.mean(cv_score))
    
    print("Model Report")
    print("MSE : ",mean_squared_error(y_test,pred))
    print("CV Score : ",cv_score)
    

In [ ]:
from sklearn.linear_model import LinearRegression
plt.figure(figsize=(10,5))
model = LinearRegression()
train(model,X,y)
coef = pd.Series(model.coef_,X.columns).sort_values()
coef.plot(kind = 'bar',title = 'Model Coefficients')